# Ingestion Step (Etapa da Ingestão)
## Table of Contents
* [Packages](#1)
* [Ingestion Utils Functions](#2)
    * [Request Authorization](#2.1)
    * [Playlist Extraction](#2.2)
    * [Track Features Extraction](#2.3)
    * [Playlist to DataFrame](#2.4)
* [Environment Variables](#3)
* [Data Extraction](#4)
    * [Request Authorization](#4.1)
    * [Playlists Definition](#4.2)
    * [Data Extraction and Preparation](#4.3)

<a class="anchor" id="1"></a>
## Packages (Pacotes)
**[EN-US]**

Packages used in the system.
* [os](https://docs.python.org/3/library/os.html): built-in module, provides a portable way of using operating system dependent functionality;
* [requests](https://pypi.org/project/requests/): is a simple HTTP library, for making HTTP requests;
* [base64](https://docs.python.org/pt-br/3.7/library/base64.html): provides functions for encoding binary data to printable ASCII characters and decoding such encodings back to binary data;
* [dotenv](https://pypi.org/project/python-dotenv/): reads key-value pairs from a .env file and can set them as environment variables;
* [pandas](https://pandas.pydata.org/): is the main package for data manipulation;
* [numpy](www.numpy.org): is the main package for scientific computing.

**[PT-BR]**

Pacotes utilizados no sistema.
* [os](https://docs.python.org/3/library/os.html): módulo integrado, fornece uma maneira portátil de usar funcionalidades dependentes do sistema operacional;
* [requests](https://pypi.org/project/requests/): é uma biblioteca HTTP simples, para fazer solicitações HTTP;
* [base64](https://docs.python.org/pt-br/3.7/library/base64.html): fornece funções para codificar dados binários em caracteres ASCII imprimíveis e decodificar essas codificações de volta para dados binários;
* [dotenv](https://pypi.org/project/python-dotenv/): lê pares de chave-valor de um arquivo .env e pode defini-los como variáveis de ambiente;
* [pandas](https://pandas.pydata.org/): é o principal pacote para manipulação de dados;
* [numpy](www.numpy.org): é o principal pacote para computação científica.

In [21]:
import os
from requests import post, get
from base64 import b64encode
from dotenv import load_dotenv
load_dotenv() # access environment variables (acessa as variáveis de ambiente)

import pandas as pd
import numpy as np

<a name="2"></a>
## Ingestion Utility Functions (Funções Utilitárias de Ingestão)
<img align='center' src='../figures/auth-client-credentials.png' style='width:800px;'>

**[EN-US]**

Utility functions for ingestion. `request_auth` function to request authorization from Spotify, `get_playlist` function to extract the tracks from a playlist and the `track_features` function to extract the features of these tracks. After that, the `playlist_to_dataframe` function transforms the data in JSON to a pandas DataFrame.

**[PT-BR]**

Funções utilitárias para a ingestão. Função `request_auth` para requisitar a autorização ao spotify, função `get_playlist` para extrair as tracks de uma playlist e a função `track_features` para extrair as features dessas tracks. Após isso, a função `playlist_to_dataframe` transforma os dados em JSON para um DataFrame pandas.

<a name="2.1"></a>
### Request Authorization (Requisita a Autorização)
**[EN-US]**

The first step is to send a POST request to the `/api/token` endpoint of the Spotify OAuth 2.0 Service. If everything goes well, you'll receive a response with a 200 OK status and the JSON data.

**[PT-BR]**

A primeira etapa é enviar uma requisição POST para o endpoint `/api/token` do serviço Spotify OAuth 2.0. Se tudo correr bem, você receberá uma resposta com o status 200 OK e os dados JSON.

In [285]:
def request_auth(client_id, client_secret):
    """
    [EN-US]
    The first step is to send a POST request to the /api/token endpoint of the Spotify OAuth 2.0 Service.
    If everything goes well, you'll receive a response with a 200 OK status and the JSON data.
    
    [PT-BR]
    A primeira etapa é enviar uma requisição POST para o endpoint /api/token do serviço Spotify OAuth 2.0.
    Se tudo correr bem, você receberá uma resposta com o status 200 OK e os dados JSON.
    
    Arguments:
        client_id -- The client ID generated after registering your application
                     (O cliente ID gerado após registrar seu aplicativo).
        client_secret -- the client secret generated after registering your application
                         (O client secret gerado após o registro do seu aplicativo).
        
    Returns:
        access_token -- An access token that can be provided in subsequent calls, for example to Spotify Web API services
                        (Um token de acesso que pode ser fornecido em chamadas subsequentes, por exemplo, para serviços Spotify Web API).
        token_type -- How the access token may be used: always "Bearer"
                      (Como o token de acesso pode ser utilizado: sempre “Bearer”).
        token_expires -- The time period (in seconds) for which the access token is valid
                         (O período de tempo (em segundos) durante o qual o token de acesso é válido).
    """
    # Base 64 encoded string that contains the client ID and client secret key (String codificada em base 64 que contém o clinet ID e a client secret key)
    b64_encoded = b64encode(f'{client_id}:{client_secret}'.encode())
    auth_b64 = str(b64_encoded, 'utf-8')
    # Setting options for authorization (Definindo as opções para a autorização)
    base_url = 'https://accounts.spotify.com/api/token'    
    
    auth_options = {
        'url': base_url,
        'headers':{
            'Authorization': 'Basic ' + auth_b64,
            'content-type': 'application/x-www-form-urlencoded'
        },
        'form':{
            'grant_type': 'client_credentials'
        },
        'json': True
    }

    # Send a post request (Enviando a requisição post)
    request = post(url=auth_options['url'], headers=auth_options['headers'], data=auth_options['form'])
    # If the request status code is 200 (Caso o status code da requisição for 200)
    if request.status_code == 200:
        json_request = request.json()
        access_token = json_request['access_token']
        token_type = json_request['token_type']
        token_expires = json_request['expires_in']
        print('Acesso autorizado e dados extraídos!')
    else:
        print('Acesso não autorizado!')
        
    return access_token, token_type, token_expires

<a name="2.2"></a>
### Playlist Extraction (Extração da Playlist)
**[EN-US]**

Get full details of the items of a playlist owned by a Spotify user.

**[PT-BR]**

Obtenha detalhes completos dos itens de uma playlist de um usuário do Spotify.

In [287]:
def get_playlist(playlist, token_type, access_token, offset=0, limit=100):
    """
    [EN-US]
    Get full details of the items of a playlist owned by a Spotify user.
    
    [PT-BR]
    Obtenha detalhes completos dos itens de uma playlist de propriedade de um usuário do Spotify.
    
    Arguments:
        playlist -- The Spotify URL of the playlist
                    (O URL do Spotify da playlist).
        token_type -- How the access token may be used: always "Bearer"
                      (Como o token de acesso pode ser utilizado: sempre “Bearer”).
        access_token -- An access token that can be provided in subsequent calls, for example to Spotify Web API services
                        (Um token de acesso que pode ser fornecido em chamadas subsequentes, por exemplo, para serviços Spotify Web API).
        offset -- The index of the first item to return. Default: 0 (the first item). Use with limit to get the next set of items
                  (O índice do primeiro item a ser retornado. Padrão: 0 (o primeiro item). Use com limit para obter o próximo conjunto de itens).
        limit -- The maximum number of items to return. Default: 100. Minimum: 1. Maximum: 100
                 (O número máximo de itens a serem retornados. Padrão: 100. Mínimo: 1. Máximo: 100).
    
    Return:
        response.json() -- Pages of tracks in JSON (Páginas de tracks em JSON).
    """
    # Turning the Spotify playlist URL into just the Playlist ID to send the get request (Transformando o Spotify URL da playlist apenas no Playlist ID para enviar a requisição get)
    playlist_id = playlist.split('/')[-1].split('?')[0]
    # Setting options for the request (Definindo as opções para a requisição)
    endpoint = f'https://api.spotify.com/v1/playlists/{playlist_id}/tracks?offset={str(offset)}&limit={str(limit)}'
    headers={
        'Authorization': token_type + ' ' + access_token
    }

    # Send a get request (Enviando a requisição get)
    response = get(url=endpoint, headers=headers)
    # If the request status code is not 200 (Caso o status code da requisição não seja 200)
    if response.status_code != 200:
        print('Error! Playlist data not extracted.')
    
    return response.json()

<a name="2.3"></a>
### Track Features Extraction (Extração das Features da Track)
<img align='center' src='../figures/track-features.png' style='width:800px;'>

**[EN-US]**

Get audio feature information for a single track identified by its unique Spotify ID.

**Note:** Spotify has removed the feature to extract the features of each track from the API.

**[PT-BR]**

Obtenha informações sobre features de áudio de uma única faixa identificada por seu ID exclusivo do Spotify.

**Nota:** o Spotify removeu o feature para extrair as features de cada track da API.

In [289]:
def track_features(track_id, token_type, access_token):
    """
    Deprecated
    [EN-US]
    Get audio feature information for a single track identified by its unique Spotify ID.
    
    [PT-BR]
    Obtenha informações sobre features de áudio de uma única faixa identificada por seu ID exclusivo do Spotify.
    
    Arguments:
        track_id -- The Spotify ID for the track (O ID do Spotify para a track).
        token_type -- How the access token may be used: always "Bearer"
                      (Como o token de acesso pode ser utilizado: sempre “Bearer”).
        access_token -- An access token that can be provided in subsequent calls, for example to Spotify Web API services
                        (Um token de acesso que pode ser fornecido em chamadas subsequentes, por exemplo, para serviços Spotify Web API).
    
    Return:
        response.json() -- Audio features for one track in JSON
                           (Features de áudio para uma track em JSON).
    """
    # Setting options for the request (Definindo as opções para a requisição)
    endpoint = f'https://api.spotify.com/v1/audio-features/{track_id}'
    headers={
        'Authorization': token_type + ' ' + access_token
    }
    
    # Send a get request (Enviando a requisição get)
    response = get(url=endpoint, headers=headers)
    # If the request status code is not 200 (Caso o status code da requisição não seja 200)
    if response.status_code != 200:
        print('Error! Track data not extracted.')
    
    return response.json()    

<a name="2.4"></a>
### Playlist to DataFrame (Playlist para DataFrame)
**[EN-US]**

Input's a playlist URL and returns a pandas DataFrame.

**[PT-BR]**

A entrada é um URL de uma playlist e retorna um DataFrame do pandas.

In [291]:
def playlist_to_dataframe(playlist_endpoint, token_type, access_token, offset=0, limit=100, label=None):
    """
    [EN-US]
    Input's a playlist URL and returns a pandas DataFrame.
    
    [PT-BR]
    A entrada é um URL de uma playlist e retorna um DataFrame do pandas.
    
    Arguments:
        playlist_endpoint -- The Spotify URL of the playlist (O URL do Spotify da playlist).
        token_type -- How the access token may be used: always "Bearer"
                      (Como o token de acesso pode ser utilizado: sempre “Bearer”).
        access_token -- An access token that can be provided in subsequent calls, for example to Spotify Web API services
                        (Um token de acesso que pode ser fornecido em chamadas subsequentes, por exemplo, para serviços Spotify Web API).
        offset -- The index of the first item to return. Default: 0 (the first item). Use with limit to get the next set of items
                  (O índice do primeiro item a ser retornado. Padrão: 0 (o primeiro item). Use com limit para obter o próximo conjunto de itens).
        limit -- The maximum number of items to return. Default: 100. Minimum: 1. Maximum: 100
                 (O número máximo de itens a serem retornados. Padrão: 100. Mínimo: 1. Máximo: 100).
        label -- The label that the playlist will be classified by. Default: None. Label 1 for good playlist and 0 for bad playlist
                 (O label que a playlist será classificada. Padrão: None. Label 1 para playlist boa e 0 para playlist ruim).
        
    Return:
        df -- Pandas DataFrame with all tracks and the features of each track in the playlist
              (DataFrame pandas com todas as tracks e as features de cada track da playlist).
    """
    # Final list that will be transformed into DataFrame (Lista final que será transformada em DataFrame)
    examples = []
    # Tracks in JSON (Tracks em JSON)
    playlist = get_playlist(playlist=playlist_endpoint, token_type=token_type, access_token=access_token, offset=offset, limit=limit)
    
    # For loop through all playlist pages (Percorrendo todas as páginas da playlist)
    for page in range(offset, playlist['total'] + 1, limit):
        # Example size on page (Tamanho de exemplo na página)
        m = len(playlist['items'])
        # For loop through each example and adding the data to the final list (Percorrendo cada exemplo e adicionando os dados à lista final)
        for i in range(m):
            # Accessing each track (Acessando cada track)
            track = playlist['items'][i]['track']
            # Track features in JSON (Features da track em JSON)
            track_feature = track_features(track['id'], token_type, access_token) # Deprecated


            # Intermediate list for append each m example (Lista intermediária para adicionar cada exemplo m)
            example = [
                track['id'],
                #track['uri'],
                track['name'],
                track['artists'][0]['name'],
                track['duration_ms'],
                track['popularity'],
                track_feature['danceability'],
                track_feature['energy'],
                track_feature['key'],
                track_feature['loudness'],
                track_feature['mode'],
                track_feature['speechiness'],
                track_feature['acousticness'],
                track_feature['instrumentalness'],
                track_feature['liveness'],
                track_feature['valence'],
                track_feature['tempo']
            ]

            # Adding to final list (Adicionando à lista final)
            examples.append(example)
        # If the page has more than 100 examples, you will read the next page
        # (Se a página tiver mais de 100 exemplos, lerá a próxima página)
        if m >= 100:
            # JSON tracks of the next offset (Tracks em JSON do próximo offset)
            playlist = get_playlist(playlist=playlist_endpoint, token_type=token_type, access_token=access_token, offset=page + limit, limit=limit)
    
    # DataFrame's features names (Nomes das features do DataFrame)
    features = ['id', 'name', 'artists', 'duration_ms', 'popularity', 'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
               'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo']
    # Transforming into DataFrame (Transformando em DataFrame)
    df = pd.DataFrame(examples, columns=features)
    
    # Setting the labels (Definindo os labels)
    if label == 0:
        df['y'] = 0
    elif label == 1:
        df['y'] = 1
    
    return df

<a name="3"></a>
## Environment Variables (Variáveis de Ambiente)
**[EN-US]**

Setting the environment variables:
* `CLIENT_ID_SPOTIFY`: the client ID generated after registering your application.
* `CLIENT_SECRET_SPOTIFY`: the client secret generated after registering your application.

**[PT-BR]**

Definindo as variáveis de ambiente:
* `CLIENT_ID_SPOTIFY`: o cliente ID gerado após registrar seu aplicativo.
* `CLIENT_SECRET_SPOTIFY`: o client secret gerado após o registro do seu aplicativo.

In [281]:
CLIENT_ID = os.environ['CLIENT_ID_SPOTIFY']
CLIENT_SECRET = os.environ['CLIENT_SECRET_SPOTIFY']

<a name="4"></a>
## Playlist Data Extraction (Extração dos Dados da Playlist)
**[EN-US]**

Requesting Spotify authorization, extracting data from Spotify tracks, transforming this data and loading the data to disk.

**[PT-BR]**

Requisitando a autorização do Spotify, extraindo os dados das tracks do spotify, transformando esses dados e carregamento os dados no disco.

<a name="4.1"></a>
### Request Authorization (Requisitando a Autorização)
Requesting Spotify Authorization (Requisitando a autorização do Spotify).

In [324]:
access_token, token_type, token_expires = request_auth(client_id=CLIENT_ID, client_secret=CLIENT_SECRET)

Acesso autorizado e dados extraídos!


In [325]:
print(f'The token expires in: {token_expires}s')

The token expires in: 3600s


<a name="4.2"></a>
### Playlists Definition (Definição das Playlists)
Setting the playlists to be extracted (Definindo as playlists que serão extraidas).

In [37]:
good_songs = 'https://open.spotify.com/playlist/6DI0NiX9bE3fIF6cEoI2zL?si=ef8a610d53f64627'
good_songs_2 = 'https://open.spotify.com/playlist/08bsg8CsImua5vzGMoiGLT?si=7ea323f1c8404b0b'
bad_songs = 'https://open.spotify.com/playlist/6IBody2iNg5TgmAeYiHYpW?si=xsEvNjbbQYqt0rs9wP3yOg'

<a name="4.3"></a>
### Data ETL. Extraction, Transform and Load (Extração, Transformação e Carregamento dos Dados)
Extracting the data from playlists and projecting the first 5 examples from the `df_good` playlist dataset (Extraindo os dados das plalists e projetando os 5 primeiros exemplos do dataset da playlist `df_good`).

In [ ]:
df_good = playlist_to_dataframe(good_songs, token_type, access_token, label=1)
df_good_2 = playlist_to_dataframe(good_songs_2, token_type, access_token, label=1)
df_bad = playlist_to_dataframe(bad_songs, token_type, access_token, label=0).drop(columns=['name'])
df_good.head()

Creating the `duration_min` feature (Criando a feature `duration_min`).

In [ ]:
df_good['duration_min'] = df_good['duration_ms'] / 60000
df_good_2['duration_min'] = df_good_2['duration_ms'] / 60000
df_bad['duration_min'] = df_bad['duration_ms'] / 60000
df_good.head()

Loading each dataset into the `./data/raw/` directory (Carregando cada dataset no diretório `./data/raw/`).

In [ ]:
df_good.to_csv('../data/raw/df_good.csv', index=False)
df_good_2.to_csv('../data/raw/df_good_2.csv', index=False)
df_bad.to_csv('../data/raw/df_bad.csv', index=False)